# Introduzione al Web Scraping con Python

In questo notebook mostreremo come effettuare Web Scraping usando due librerie leggere e potenti: `requests` e `BeautifulSoup`.

Eviteremo strumenti complessi come `Selenium`, così che il codice sia eseguibile anche su ambienti come [Binder](https://mybinder.org).

Obiettivo: Estrarre titoli e link delle notizie dalla homepage di un sito web semplice e statico.

## 1. Cos'è il Web Scraping?

Il Web Scraping è la tecnica di estrazione automatica di contenuti da pagine web.

### Componenti principali:
- **HTTP client**: per inviare richieste e ricevere le pagine HTML (es. `requests`)
- **Parser HTML**: per analizzare e navigare la struttura del documento HTML (es. `BeautifulSoup`)

**Attenzione**: Prima di effettuare scraping su un sito web, verifica i suoi termini di servizio e il file `robots.txt`.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

## 2. Esempio base: Estrazione titoli da un sito web

Come esempio useremo il sito [https://quotes.toscrape.com/](https://quotes.toscrape.com/), pensato proprio per fare pratica con lo scraping.

Vogliamo ottenere una lista delle citazioni presenti nella homepage.

In [ ]:
url = "https://quotes.toscrape.com/"
response = requests.get(url)

response.status_code

### Analisi della risposta

Se il codice di stato è `200`, la richiesta ha avuto successo e possiamo analizzare il contenuto HTML.

In [ ]:
html = response.text
soup = BeautifulSoup(html, "html.parser")

## 3. Navigare il DOM con BeautifulSoup

Analizziamo la struttura del documento per trovare dove sono contenute le citazioni. Ogni citazione è in un elemento `<div class="quote">`.

In [ ]:
quotes = soup.find_all("div", class_="quote")
len(quotes)

## 4. Estrazione dei dati

Per ogni citazione, estraiamo:
- Il testo della citazione
- L'autore
- Le tag associate

In [ ]:
for quote in quotes[:3]:
    text = quote.find("span", class_="text").get_text()
    author = quote.find("small", class_="author").get_text()
    tags = [tag.get_text() for tag in quote.find_all("a", class_="tag")]
    print(f"CITAZIONE: {text}")
    print(f"  AUTORE: {author}")
    print(f"  TAG: {tags}")
    print()

## 5. Scraping di più pagine

Il sito ha una paginazione classica con un link `Next`.

Vediamo come fare scraping su più pagine navigando automaticamente finché esiste la pagina successiva.

In [ ]:
quotes_data = []
url = "https://quotes.toscrape.com/"

while url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    for quote in soup.find_all("div", class_="quote"):
        text = quote.find("span", class_="text").get_text()
        author = quote.find("small", class_="author").get_text()
        tags = [tag.get_text() for tag in quote.find_all("a", class_="tag")]
        quotes_data.append({
            "text": text,
            "author": author,
            "tags": tags
        })

    next_btn = soup.find("li", class_="next")
    if next_btn:
        next_link = next_btn.find("a")["href"]
        url = "https://quotes.toscrape.com" + next_link
    else:
        url = None

In [ ]:
len(quotes_data)

In [ ]:
quotes_data[:2]

## Salvataggio dei dati in CSV

Salviamo la lista di citazioni estratte in un file CSV, per poterla usare o analizzare successivamente.

In [ ]:
csv_filename = "quotes.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["text", "author", "tags"])
    writer.writeheader()
    for row in quotes_data:
        row["tags"] = ", ".join(row["tags"])
        writer.writerow(row)
print(f"Dati salvati su {csv_filename}")


## Esempio di autenticazione Basic HTTP

Se un sito è protetto da autenticazione Basic HTTP, puoi passare le credenziali con `requests`.

Per esempio, se un sito richiede username e password:

- user: `admin`
- password: `password123`

Puoi fare così:

In [ ]:
auth_url = "https://httpbin.org/basic-auth/admin/password123"
response_auth = requests.get(auth_url, auth=("admin", "password123"))

if response_auth.status_code == 200:
    print("Autenticazione riuscita")
    data = response_auth.json()
    print(data)
else:
    print(f"Autenticazione fallita con codice {response_auth.status_code}")


## 6. Analisi del file robots.txt

Il file `robots.txt` è uno standard usato dai siti web per indicare ai bot (crawler, scraper) quali pagine possono o non possono essere visitate.

### Perché è importante leggere `robots.txt`?
- Rispettare le regole del sito web (etica del scraping)
- Evitare di essere bloccati o bannati
- Comprendere quali sezioni sono accessibili ai bot

---

Un esempio di file `robots.txt` può essere:

User-agent: *
Disallow: /private/
Allow: /public/

Significa che tutti i bot (`*`) non possono visitare `/private/`, ma possono visitare `/public/`.

In [ ]:
def fetch_robots_txt(base_url):
    if not base_url.endswith('/'):
        base_url += '/'
    robots_url = base_url + "robots.txt"
    response = requests.get(robots_url)
    if response.status_code == 200:
        return response.text
    else:
        return None

site = "https://www.wikipedia.org"
robots_txt = fetch_robots_txt(site)
print("robots.txt content:\n")
print(robots_txt)


### Interpretazione semplificata del file robots.txt

Vediamo ora come estrarre le regole principali:

- **User-agent**: definisce a quali bot si applicano le regole successive
- **Disallow**: percorsi vietati ai bot specificati
- **Allow**: percorsi esplicitamente permessi

Qui mostreremo un parser semplice che stampa le regole per tutti i bot (`*`).

In [ ]:
def parse_robots_txt(robots_txt):
    lines = robots_txt.splitlines()
    user_agent = None
    rules = {"Disallow": [], "Allow": []}
    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if line.lower().startswith("user-agent:"):
            user_agent = line.split(":", 1)[1].strip()
        elif line.lower().startswith("disallow:") and user_agent == "*":
            path = line.split(":", 1)[1].strip()
            if path:
                rules["Disallow"].append(path)
        elif line.lower().startswith("allow:") and user_agent == "*":
            path = line.split(":", 1)[1].strip()
            if path:
                rules["Allow"].append(path)
    return rules

if robots_txt:
    rules = parse_robots_txt(robots_txt)
    print("Regole per User-agent '*':")
    print("Disallow:", rules["Disallow"])
    print("Allow:", rules["Allow"])
else:
    print("Nessun file robots.txt trovato o non accessibile.")

## Conclusioni

In questo notebook abbiamo visto come:

- Fare scraping semplice con `requests` e `BeautifulSoup`
- Estrarre dati e salvarli in CSV
- Gestire autenticazione Basic HTTP
- Scaricare e interpretare il file `robots.txt` per rispettare le regole di scraping

Leggere e rispettare il file `robots.txt` è fondamentale per uno scraping etico e sostenibile.

Per pagine con contenuti caricati dinamicamente via JavaScript, si può ricorrere a strumenti più avanzati come Selenium o Playwright, che permettono di eseguire un browser reale o headless e interagire con la pagina..